# Libraries

In [ ]:
import numpy as np
import cv2
import os
import random
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from skimage.segmentation import find_boundaries
from sklearn.metrics import f1_score

# Torch libraries
import torch
import torchvision.models as models
from torchvision import transforms
from torchvision.transforms import transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision.transforms import transforms
import torch.nn.functional as F

## Random Seed

In [ ]:
seed_value = 42

np.random.seed(seed_value)  # NumPy
torch.manual_seed(seed_value)  # PyTorch CPU
torch.cuda.manual_seed_all(seed_value)  # PyTorch GPU

# Data Manager

In [ ]:
def equalise_image(image):
   gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
   equalized_image = cv2.equalizeHist(gray_image)
   equalized_image = cv2.cvtColor(equalized_image, cv2.COLOR_GRAY2BGR)

   return equalized_image

Defines a custom dataset class

In [ ]:
class CustomDataset(Dataset):
   def __init__(self, images, labels, transform=None):
      self.images = images
      self.labels = labels
      self.transform = transform

   def __len__(self):
      return len(self.images)

   def __getitem__(self, idx):
      image = self.images[idx]
      imageResize = cv2.resize(image, (299, 299))

      label = self.labels[idx]
      if self.transform:
         imageResize = self.transform(imageResize)
      return imageResize, label

We also have the choice to equalise the images (unfortunately only in black and white).

In [ ]:
def load_images_with_masks(image_directory, mask_directory):
    images = []
    labels = []
    for image_filename in os.listdir(image_directory):
        if image_filename.endswith(".tif"):
            image_filepath = os.path.join(image_directory, image_filename)
            mask_filename = image_filename + '.png'
            mask_filepath = os.path.join(mask_directory, mask_filename)
            try:
                image = cv2.imread(image_filepath)
				        # image = equalise_image(image)
                mask = cv2.imread(mask_filepath, cv2.IMREAD_GRAYSCALE)
                images.append(image)
                labels.append(mask)
            except Exception as e:
                print(f"Error loading image {image_filename} or mask: {e}")
    return np.array(images), np.array(labels)

In [ ]:
def extract_subimages(images, labels, removeIndex, subimage_size=64, step_size=8):
    subimages = []
    sublabels = []
    for i in range(len(images)):
        if i != removeIndex: #skip the image used for testing
            image = images[i]
            label = labels[i]
            height, width = image.shape[:2]
            for y in range(0, height - subimage_size + 1, step_size):
                for x in range(0, width - subimage_size + 1, step_size):
                    subimage = image[y:y+subimage_size, x:x+subimage_size]
                    sublabel = label[y:y+subimage_size, x:x+subimage_size]
                    if np.all(sublabel == 0) or np.all(sublabel == 255):
                        subimages.append(subimage)
                        sublabels.append(sublabel[0][0])  # Assuming all values are the same in the sublabel
    return np.array(subimages), np.array(sublabels)

In [ ]:
def load_data(k=100):

  image_directory = "neuroendocrine_/images"
  mask_directory = "neuroendocrine_/masks"

  images, labels = load_images_with_masks(image_directory, mask_directory)

  return images, labels

In [ ]:
def load_train_data(images, labels):

  train_images, train_labels = extract_subimages(images[:-1], labels, 29)
  del images

  # labels should be 0 or 1
  train_labels[train_labels == 255] = 1

  print("Shape of the full train_images array:", train_images.shape)
  print("Shape of the full train_labels array:", train_labels.shape)

  return train_images, train_labels

In [ ]:
def load_val_data(images, labels, k=100):

  val_images, val_labels = extract_subimages(images[-1:], labels, -1)
  del labels

  val_images = val_images[:k]
  val_labels = val_labels[:k]

  # labels should be 0 or 1
  val_labels[val_labels == 255] = 1

  print("Shape of the val_labels array:", val_images.shape)
  print("Shape of the val_labels array:", val_labels.shape)

  return val_images, val_labels

In [ ]:
def get_indices(labels):

  # Assuming train_images and train_labels are your training data
  # Calculate the indices of each class
  class_0_idxs = np.where(labels == 0)[0]
  class_1_idxs = np.where(labels == 1)[0]

  print(f"{len(class_0_idxs)} non-tumor images")
  print(f"{len(class_1_idxs)} tumor images")

  return class_0_idxs, class_1_idxs

In [ ]:
def setup_transforms(augment=False):

  if augment == True:
    transform = transforms.Compose([
      transforms.ToPILImage(),
      transforms.RandomHorizontalFlip(),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      ])
  else:
    transform = transforms.Compose([
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      ])

  return transform


In [ ]:
def create_full_dataloader(batch_size=100, k=100, augment=False):

  images, labels = load_data(k=k)

  train_images, train_labels = load_train_data(images, labels)
  val_images, val_labels = load_val_data(images, labels, k)

  del images, labels

  transform = setup_transforms(augment)

  # Create the undersampled dataset
  train_dataset = CustomDataset(train_images, train_labels, transform=transform)
  val_dataset = CustomDataset(val_images, val_labels, transform=transform)

  del train_images
  del train_labels

  # Create the data loader for the undersampled dataset

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

  del train_dataset

  return train_loader, val_loader

In [ ]:
def create_balanced_dataloader(batch_size=100, k=100, augment=False):

  images, labels = load_data(k=k)

  train_images, train_labels = load_train_data(images, labels)
  val_images, val_labels = load_val_data(images, labels, k)

  del images, labels

  class_0_idxs, class_1_idxs = get_indices(train_labels)

  # Determine the size of the minority class
  minority_class_size = len(class_0_idxs)

  sel_class_0_idxs = class_0_idxs
  # Randomly sample the same number of samples from the majority class
  sel_class_1_idxs = np.random.choice(class_1_idxs, minority_class_size, replace=False)

  sel_idxs = np.concatenate([sel_class_0_idxs, sel_class_1_idxs])

  del class_0_idxs, class_1_idxs

  # Shuffle the indices to mix the samples from both classes
  np.random.shuffle(sel_idxs)

  # Use the undersampled indices to create a new balanced dataset
  train_images = train_images[sel_idxs]
  train_labels = train_labels[sel_idxs]

  del sel_idxs

  print("Shape of the selected train_images array:", train_images.shape)
  print("Shape of the selected train_labels array:", train_labels.shape)

  transform = setup_transforms(augment)

  # Create the undersampled dataset
  train_dataset = CustomDataset(train_images, train_labels, transform=transform)
  val_dataset = CustomDataset(val_images, val_labels, transform=transform)

  del train_images
  del train_labels

  # Create the data loader for the undersampled dataset
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

  del train_dataset

  return train_loader, val_loader

In [ ]:
def create_custom_dataloader(batch_size=100, n=60000, k=100, augment=False):

  images, labels = load_data(k=k)

  train_images, train_labels = load_train_data(images, labels)
  val_images, val_labels = load_val_data(images, labels, k)

  del images, labels

  class_0_idxs, class_1_idxs = get_indices(train_labels)
  num_idxs = len(train_labels)
  num_0_idxs = len(class_0_idxs)
  num_1_idxs = len(class_1_idxs)
  # Define weights for each sample
  sample_weights = np.where(train_labels==0, num_idxs/num_0_idxs, num_idxs/num_1_idxs)

  # Create a WeightedRandomSampler
  sampler = WeightedRandomSampler(weights=sample_weights, num_samples=n, replacement=True)

  transform = setup_transforms()

  # Create the undersampled dataset
  train_dataset = CustomDataset(train_images, train_labels, transform=transform)
  val_dataset = CustomDataset(val_images, val_labels, transform=transform)

  del train_images
  del train_labels

  # Create the data loader for the undersampled dataset
  train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
  val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

  # TODO add final dataloader size

  del train_dataset

  return train_loader, val_loader

In [ ]:
def get_images(dataset_path, test=False):

   suff = None

   if test == False:
      suff = "images"
   else:
      suff = "masks"

   images_dir_path = Path(f"{dataset_path}/{suff}")

   # List all files in the folder
   images_paths = sorted(list(images_dir_path.iterdir()))

   return images_paths

# Models Manager

In [ ]:
def import_model(model_name):
  if model_name == "inceptionv3":
    # Load InceptionV3 model pretrained on ImageNet
    model = models.inception_v3(weights='DEFAULT')
  elif model_name == "AlexNet":
    # Load AlexNet model pretrained on ImageNet
    model = models.alexnet(weights='DEFAULT')

  return model

In [ ]:
def setup_model(model, device):
  # Set the model to evaluation mode
  model.eval()

  num_classes = 2  # Assuming CustomDataset has a 'classes' attribute

  if isinstance(model, models.inception.Inception3):
    # Freeze all the parameters
    for param in model.parameters():
      param.requires_grad = False

    # Modify the last layer to fit your number of classes
    model.fc = nn.Sequential(
        nn.Linear(model.fc.in_features, 512),
        nn.ReLU(),
        nn.Linear(512, 128),
        nn.ReLU(),
        nn.Linear(128, 32),
        nn.ReLU(),
        nn.Linear(32, num_classes)
        )

  elif isinstance(model, models.AlexNet):
    model.classifier.add_module('7', nn.ReLU())
    model.classifier.add_module('8', nn.Linear(1000, 512))
    model.classifier.add_module('9', nn.ReLU())
    model.classifier.add_module('10', nn.Linear(512, 128))
    model.classifier.add_module('11', nn.ReLU())
    model.classifier.add_module('12', nn.Linear(128, 32))
    model.classifier.add_module('13', nn.ReLU())
    model.classifier.add_module('14', nn.Linear(32, 2))

  # Move the model to the GPU
  model = model.to(device)

In [ ]:
def setup_training(model):

	if isinstance(model, models.inception.Inception3):
		model.fc.train()

	elif isinstance(model, models.AlexNet):
		model.train()

In [ ]:
def validate(model, criterion, dataloader, device):

  val_loss = 0.0
  correct_predictions = 0
  total_samples = 0

  for inputs, labels in dataloader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    val_loss += loss.item() * inputs.size(0)
    _, predicted = torch.max(outputs, 1)
    correct_predictions += (predicted == labels).sum().item()
    total_samples += labels.size(0)

  val_loss /= total_samples
  accuracy = correct_predictions / total_samples

  print(f'Validation Loss: {val_loss:.4f}, Accuracy: {accuracy:.4f}')

### Training loop

In [ ]:
def training_loop(model, optimizer, loss_fn, train_dataloader, val_dataloader, device,
                  num_epochs, max_train, print_every, losses):

  for epoch in range(num_epochs):

    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_dataloader, 1):

      # Fetch inputs and labels
      inputs, labels = inputs.to(device), labels.to(device)

      # Initialise gradients
      optimizer.zero_grad()

      # Feedforward
      outputs = model(inputs)

      # Backpropagate
      loss = loss_fn(outputs, labels)
      loss.backward()

      # Update parameters
      optimizer.step()

      # Append loss
      running_loss += loss.item() * inputs.size(0)

      # Print average loss every print_every iterations
      if i % print_every == 0:
        epoch_loss = running_loss / (print_every * len(inputs))
        losses.append(epoch_loss)
        print(f"Iteration [{i}/{len(train_dataloader)}], Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")
        running_loss = 0.0
        validate(model, loss_fn, val_dataloader, device)
        print("------------------")

      # Iteration limit
      if i >= max_train:
        break

    print("==========================")

  return losses

# TODO move inside train function

In [ ]:
def train(model, optimizer, loss_fn, dataloader, val_dataloader, device,
          num_epochs=1, max_train=200, print_every=10, losses=[]):

  setup_training(model)

  training_loop(model, optimizer, loss_fn, dataloader, val_dataloader, device, num_epochs, max_train, print_every, losses)

In [ ]:
def load_params(model, local_path, project_path, device):

  model_path = None
  params_path = None

  if isinstance(model, models.inception.Inception3):

    model_path = "inception_v3"
    params_path = "incv3_params.pth"

  elif isinstance(model, models.AlexNet):

    model_path = "AlexNet"
    params_path = "AlexNet_params.pth"

  full_path = f"{local_path}/{params_path}"

  # Load the saved dictionary into your model

  try:
    state_dict = torch.load(full_path, map_location=device)
    model.load_state_dict(state_dict)
  except:
    print("WARNING: default ImageNet weights have been loaded.")

In [ ]:
def save_params(model, local_path):

  params_path = None

  if isinstance(model, models.inception.Inception3):
    params_path = "incv3_params.pth"

  elif isinstance(model, models.AlexNet):
    params_path = "AlexNet_params.pth"

  # Save the model state
  torch.save(model.state_dict(), f"{local_path}/{params_path}")

In [ ]:
def predict(model, device, image):

  testImage = image

  subimage_size = 64
  step_size = 8

  height, width = testImage.shape[:2]
  outputHeight = height - height % step_size
  outputWidth = width - width % step_size
  tumorCount = np.zeros((outputHeight, outputWidth))
  count = np.zeros((outputHeight, outputWidth))

  # Move model to GPU
  model.eval()

  for row in range(0, height - subimage_size + 1, step_size):
    if row % 50 == 0:
      print('Row ' + str(row) + '/ ' + str(height))
    for col in range(0, width - subimage_size + 1, step_size):
      subimage = testImage[row:row+subimage_size, col:col+subimage_size]

      # Prepare subimage for InceptionV3
      # Resize the subimage using OpenCV
      resized_subimage = cv2.resize(subimage, (299, 299))

      # Define the transformations
      transform = transforms.Compose([
          transforms.ToTensor(),  # Convert image to tensor
          transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
          ])

      # Apply the transformations and move to GPU
      transformed_subimage = transform(resized_subimage).to(device)

      # Compute output on GPU
      with torch.no_grad():
        output = model(transformed_subimage.unsqueeze(0))
        label = torch.argmax(output)

      # Write to matrix
      if label == 1:
        tumorCount[row:row+subimage_size, col:col+subimage_size] += 1
      count[row:row+subimage_size, col:col+subimage_size] += 1

  # Calculate average tumor occurrence per submatrix
  avg = np.divide(tumorCount, count)

  return avg

## Custom Loss Functions

In [ ]:
def focal_loss(alpha, gamma):

  def _focal_loss(outputs, labels):

    # Convert labels to int64 type
    labels = labels.long()

    # Compute class weights based on the frequency of each class in the labels
    class_counts = torch.bincount(labels)

    class_weights = (1 / class_counts.float()).clone().detach()

    # Normalize the class weights
    class_weights /= class_weights.sum()

    # Compute softmax along the class dimension
    input_softmax = F.softmax(outputs, dim=1)
    # Gather the probabilities of the true class
    p_t = torch.gather(input_softmax, 1, labels.view(-1, 1))
    # Compute binary cross entropy loss
    bce_loss = F.cross_entropy(outputs, labels, reduction='none')
    # Compute focal loss
    focal_loss = alpha * (1 - p_t) ** gamma * bce_loss
    return torch.mean(focal_loss)

  return _focal_loss

# Evaluation Manager

In [ ]:
def get_pred(model, local_path, project_path, img_idx):

  if isinstance(model, models.inception.Inception3):
    model_name = "inception_v3"

  elif isinstance(model, models.AlexNet):
    model_name = "AlexNet"

  if os.path.exists(f"avg_{img_idx}.png"):
    avg_path = f"{local_path}/avg_{img_idx}.png"
    print("avg image found in the runtime.")
  else:
    print("No selected avg image. Loading not succeeded!")
    return None

  avg = cv2.imread(avg_path)

  # Convert the image to grayscale
  avg = cv2.cvtColor(avg, cv2.COLOR_BGR2GRAY)
  avg = avg/255

  return avg

In [ ]:
def beautify_output(avg):
  printAvg = avg*255
  return printAvg

In [ ]:
def threshold_output(avg, thr=0.5):
  modelGuess = np.where(avg >= thr, 255, 0).astype(np.uint8)[:1283, :2040]
  return modelGuess

In [ ]:
def visualise_output(printable):
  # Displaying the numpy array as grayscale
  plt.imshow(printable, cmap='gray', vmin=0, vmax=255)  # Specify vmin and vmax
  plt.axis('off')  # Turn off axis
  plt.show()

In [ ]:
def save_pred(avg, img_idx):
  if avg is not None:
    cv2.imwrite(f'avg_{img_idx}.png', avg)
  else:
    print("No avg has been provided. Saving not succeeded!")

In [ ]:
def save_thr_pred(thr_pred, img_idx):
  if thr_pred is not None:
    cv2.imwrite(f'pred_image_{img_idx}.png', thr_pred)
  else:
    print("No thr_pred has been provided. Saving not succeeded!")

In [ ]:
def visualise_comparison(thr_output, testMask):

	combined_matrix = 2 * thr_output/255 + testMask[:,:,0]/255

	# Define colors for each case
	colors = ['black', 'green', 'red', 'white']
	color_labels = ['correct non tumor', 'false positive', 'false negative', 'correct tumor']

	# Plot the combined matrix
	plt.figure(figsize=(10, 8))
	plt.imshow(combined_matrix, cmap=plt.cm.colors.ListedColormap(colors))

	# Create a separate legend outside the plot
	legend_handles = [plt.Rectangle((0,0),1,1, color=color) for color in colors]
	plt.legend(legend_handles, color_labels, loc='upper left', bbox_to_anchor=(1, 1), facecolor='dimgrey')

	plt.show()

In [ ]:
def pixel_accuracy(y_true, y_pred):

  return accuracy_score(y_true.flatten(), y_pred.flatten())

def pixel_precision(y_true, y_pred):

  true_positives = np.sum(np.logical_and(y_true, y_pred))
  false_positives = np.sum(np.logical_and(np.logical_not(y_true), y_pred))

  if true_positives + false_positives == 0:
    return 0.0
  else:
    return true_positives / (true_positives + false_positives)

def pixel_sensitivity(y_true, y_pred): # Also known as RECALL

  true_positives = np.sum(np.logical_and(y_true, y_pred))
  false_negatives = np.sum(np.logical_and(y_true, np.logical_not(y_pred)))

  if true_positives + false_negatives == 0:
    return 0.0
  else:
    return true_positives / (true_positives + false_negatives)

def pixel_specificity(y_true, y_pred):
	true_negatives = np.sum(np.logical_and(np.logical_not(y_true), np.logical_not(y_pred)))
	false_positives = np.sum(np.logical_and(np.logical_not(y_true), y_pred))

	if true_negatives + false_positives == 0:
		return 0.0
	else:
		return true_negatives / (true_negatives + false_positives)


def intersection_over_union(y_true, y_pred):

  intersection = np.logical_and(y_true, y_pred)
  union = np.logical_or(y_true, y_pred)
  return np.sum(intersection) / np.sum(union)

def dice_coefficient(y_true, y_pred):

  intersection = np.logical_and(y_true, y_pred)

  return 2.0 * np.sum(intersection) / (np.sum(y_true) + np.sum(y_pred))

def f1_score(y_true, y_pred):
	precision = pixel_precision(y_true, y_pred)
	recall = pixel_specificity(y_true, y_pred)

	if precision + recall == 0:
		return 0.0
	else:
		return 2.0 * (precision * recall) / (precision + recall)

def boundary_f1_score(y_true, y_pred):
	# Find boundaries of the ground truth and predicted segmentation masks
	true_boundaries = find_boundaries(y_true)
	pred_boundaries = find_boundaries(y_pred)

	# Flatten the boundary masks
	true_boundaries_flat = true_boundaries.flatten()
	pred_boundaries_flat = pred_boundaries.flatten()

	# Compute F1 score for boundary detection
	f1_boundary = f1_score(true_boundaries_flat, pred_boundaries_flat)

	return f1_boundary

In [ ]:
def get_true_labels(mask, tumor=True):
  y_true = (mask[:,:,0]/255).astype(bool)

  if tumor == False:
    y_true = np.logical_not(y_true)

  return y_true

def get_pred_labels(mask, tumor=True):
  y_pred = (mask/255).astype(bool)

  if tumor == False:
    y_pred = np.logical_not(y_pred)

  return y_pred

In [ ]:
def show_metrics(acc, prec, spec, sens, iou, dice, f1_sc, bound_f1_sc):

  # Organize the metrics into a list of lists
  metrics_table = [
    ["Pixel Accuracy", f"{acc:0.4f}"],
    ["Pixel Precision", f"{prec:0.4f}"],
    ["Pixel Specificity (Recall)", f"{spec:0.4f}"],
    ["Pixel Sensitivity", f"{sens:0.4f}"],
    ["IoU", f"{iou:0.4f}"],
    ["Dice Coeff.", f"{dice:0.4f}"],
    ["F1 Score.", f"{f1_sc:0.4f}"],
    ["Boundary F1 Score.", f"{bound_f1_sc:0.4f}"],
  ]

  # Display the metrics table
  print(tabulate(metrics_table, headers=["Metric", "Value"], tablefmt="grid"))

In [ ]:
def conf_matrix(y_true, y_pred, normalise=None):

  conf_matrix = confusion_matrix(y_true.flatten(), y_pred.flatten(), normalize=normalise)
  return conf_matrix

def format_conf_matrices(conf_matrix, conf_norm_matrix):

  fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(14, 8))

  text = """
  - 0: non-tumor pixel
  - 1: tumor pixel
  """

  fig.text(0.5, 0.90, text, ha='center', fontsize=16, color='black')

  sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", linewidths=.5, square=True, ax=axs[0])
  axs[0].set_title("Confusion matrix")
  axs[0].set_xlabel("Predicted label")
  axs[0].set_ylabel("True label")

  sns.heatmap(conf_norm_matrix, annot=True, fmt=".2%", cmap="Blues", linewidths=.5, square=True, ax=axs[1])
  axs[1].set_title("Confusion normalised matrix")
  axs[1].set_xlabel("Predicted label")
  axs[1].set_ylabel("True label")

  return fig

In [ ]:
def visualize_bordered_mask(testImage, modelGuess, radius):
  # Define the disk structuring element
  radius = 2
  disk_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*radius+1, 2*radius+1))

  # Apply erosion using the disk structuring element
  eroded_mask = cv2.erode(modelGuess, disk_kernel)

  tumorBorder = modelGuess - eroded_mask

  map_rgb = cv2.cvtColor(tumorBorder, cv2.COLOR_GRAY2RGB)
  alpha = 0.5  # Adjust the transparency of the overlaid image
  overlay = cv2.addWeighted(testImage[:1283, :2040], alpha, map_rgb, 1 - alpha, 0)

  plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.show()

# Notebook Control

## Training/Loading

Control the training/loading and saving functionalities:

- **training**: if yes, performs the computational training of the model. Otherwise, downloads and loads the already trained parameters.
- **save**: if yes, saves the current state of the model (regardless of the fact that the training has been performed or not).

**Important**: write operations are performed even if files are already present in the runtime. Hence, old files are over-written.

In [ ]:
training = False
save = False # regardless of the actual training

## Prediction

Control the prediction/loading and writing functionalities:

- **pred**: if yes, performs the computational prediction of a selected image's segmentation. Otherwise, loads the file avg.png from the Colab runtime (if it is not present, it downloads it from th GitHub project's repo).
- **avg_write**: if yes, write to the Colab runtime the raw output of the model based on the chosen image (regardless of the fact that the computation has been performed or not).
- **output_write**: if yes, write to the Colab runtime the thresholded output of the model based on the chosen image (regardless of the fact that the computation has been performed or not).

**Important**: write operations are performed even if files are already present in the runtime. Hence, old files are over-written.

In [ ]:
compute = False # If true, compute the mask prediction of a chosen image; otherwise, consider the already pre-computed prediction.
avg_write = compute = False # If true, compute the mask prediction of a chosen image; otherwise, consider the already pre-computed prediction.
conf_matr_write = False # Write the confusion matrices figure to Colab.
output_write = False # Write the final output photo to Colab.

In [ ]:
img_idx = 29 # index of image chosen for evaluation (loaded avg.png refers to 29)

# Resources Import

In [ ]:
!git clone https://github.com/Luca-Olivieri/NAML_project.git

### Resources paths

In [ ]:
local_path = "/content/"
project_path = local_path+"NAML_project"
dataset_path = local_path+"neuroendocrine_"

# Model Import

Import one of the pre-trained models.

Available models:

- Inception v3
- AlexNet

In [ ]:
# model_name = "inceptionv3"
model_name = "AlexNet"

Select version:

In [ ]:
model = import_model(model_name)

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
setup_model(model, device)

# Data Ingestion

Download dataset from GitHub:

In [ ]:
!git clone https://github.com/cialab/neuroendocrine_

# Training/Loading

Dataset is very unbalanced, so I create a dataset containing 50% tumor and 50% non tumor subimages.
I tried to trained on the original unbaiased dataset and the network pretty much always predicted tumor (because in training 90% of subimages are tumor)

In [ ]:
train_loader = None
val_loader = None

In [ ]:
if training == True:

  batch_size = 100

  del train_loader, val_loader
  torch.cuda.empty_cache()

  train_loader, val_loader = create_custom_dataloader(batch_size, n=200_000, k=100, augment=True)
  # train_loader, val_loader = create_full_dataloader(batch_size, k=100, augment=True)
  # train_loader, val_loader = create_balanced_dataloader(batch_size, k=100, augment=True)

	# n denotes the number of 64x64 images.
	# p denote the percentage of non-tumor images.
  # k denotes the number of validation 64x64 images.

In [ ]:
if training == True:

	# Define your optimiser:
	optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
	# optimizer = optim.Adam(model.parameters(), lr=0.00075)
	# optimizer = optim.AdamW(model.parameters(), lr=0.00075)

	# Define your loss function:
	# loss_fn = nn.CrossEntropyLoss()
	loss_fn = nn.CrossEntropyLoss(weight=torch.tensor([1.75, 1], dtype=torch.float).to(device))
	# loss_fn = focal_loss(alpha=0.4, gamma=1.5) # alpha = 1/(1+w_0)

	losses = []

	# Proceed with the training loop
	train(model, optimizer, loss_fn, train_loader, val_loader, device,
								num_epochs=2, max_train=1e8, print_every=10, losses=losses)
else:

	# Load custom pre-trained parameters
	load_params(model, local_path, project_path, device)

In [ ]:
# If training has performed, plot the
if training == True:
  plt.plot(losses)

### Parameters saving

In [ ]:
if save == True:
   save_params(model, local_path)

# Testing

### Target visualisation

Available photos:

In [ ]:
images_paths = get_images(dataset_path, test=False)
masks_paths = get_images(dataset_path, test=True)

In [ ]:
# images_paths

Considered photo:

In [ ]:
images_paths[img_idx], masks_paths[img_idx]

In [ ]:
testImage = cv2.imread(str(images_paths[img_idx]))
testMask = cv2.imread(str(masks_paths[img_idx]))

plt.imshow(testImage)
plt.imshow(testMask, alpha=0.5)

### Mask prediction

In [ ]:
if compute == True:
  avg = predict(model, device, testImage)

else:
  avg = get_pred(
    model, local_path, project_path, img_idx)

### Mask predition saving

In [ ]:
if avg_write == True:
  save_pred(avg, img_idx)

### Visualisations

Raw output of the segmentation:

In [ ]:
raw_output = beautify_output(avg)
visualise_output(raw_output)

Thresholded output:

In [ ]:
thr = 0.5 # threshold

thr_output = threshold_output(avg, thr)
visualise_output(thr_output)

Visualisation of the prediction quality:

In [ ]:
visualise_comparison(thr_output, testMask)

Prediction saving:

In [ ]:
if output_write == True:
	save_thr_pred(thr_output, img_idx)

# Evaluation

### Metrics

In [ ]:
tumor = True

In [ ]:
y_true = get_true_labels(testMask, tumor)
y_pred = get_pred_labels(thr_output, tumor)

Compute metrics:
- Accuracy
- ...

In [ ]:
acc = pixel_accuracy(y_true, y_pred)
prec = pixel_precision(y_true, y_pred)
spec = pixel_specificity(y_true, y_pred)
sens = pixel_sensitivity(y_true, y_pred)
iou = intersection_over_union(y_true, y_pred)
dice = dice_coefficient(y_true, y_pred)
f1_sc = f1_score(y_true, y_pred)
bound_f1_sc = f1_score(y_true, y_pred)

Show metric table:

In [ ]:
show_metrics(acc, prec, spec, sens, iou, dice, f1_sc, bound_f1_sc)

### Confusion matrix:

In [ ]:
type(conf_matrix)

In [ ]:
conf_m = conf_matrix(y_true, y_pred)
conf_norm_m = conf_matrix(y_true, y_pred, normalise="true")

In [ ]:
fig = format_conf_matrices(conf_m, conf_norm_m)

In [ ]:
if conf_matr_write == True:
  fig.savefig(f"conf_matr_{img_idx}.png")

### Visualisation

Visualisation of prediction border onto input image:

In [ ]:
visualize_bordered_mask(testImage, thr_output, radius=2)